In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

from skimage.morphology import binary_erosion
from scipy.ndimage import binary_fill_holes
import Scripts.BT_Height_Helpers as BTH
from skimage.measure import label
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from satpy import Scene
%matplotlib notebook
from glob import glob
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Folder containing satellite data
indir_sat = 'E:/data/HIM/'

# Filename of CSV file with ECMWF profile
infile_ecm = "ECM.csv"

# Directory for output
odir_top = 'C:/Users/EUMETCAST#/Desktop/Volcano/'
height_subdir = f'{odir_top}/Height_Out/'
pres_subdir = f'{odir_top}/Pres_Out/'
label_subdir = f'{odir_top}/Label/'
bt_subdir = f'{odir_top}/BTemp/'

# Band to process: B13 is 10.3 micron window channel
procband = 'B13'

# Window size in pixels for finding label of central plume
dpix = 10

# Set brightness temperature threshold (in K) for filtering
bt_thresh = 250

# Set secondary threshold for tropopause plume
bt_thresh_trop = 210

# This sets up a bounding box around the volcano in which we work
c_lat = -20.32
c_lon = -175.23
deller = 7.5
bbox = (c_lon - deller,
        c_lat - deller, 
        c_lon + deller,
        c_lat + deller)

bbox = (-178.75, -24.5, -171.5, -17.3)

In [3]:
# Get ECMWF data from file
ecm_dt = np.loadtxt(open(infile_ecm, "rb"), delimiter=",", skiprows=1)
# Flip the array so lowest altitude is in position zero
ecm_dt = np.flipud(ecm_dt)

In [4]:
# Find data files to process
firstfiles = glob(f'{indir_sat}/*{procband}*S07*.DAT')

In [5]:
# Loop over files and process
for inf in tqdm(firstfiles):
    # Find date time string to load remaining files
    pos = inf.find('_B13_')
    dtstr = inf[pos-13:pos]
    curfiles = glob(f'{indir_sat}/*{dtstr}*{procband}*.DAT')
    
    # Make satpy scene
    scn = Scene(curfiles, reader='ahi_hsd')
    scn.load([procband])
    scn2 = scn.crop(ll_bbox=bbox)
    
    # Get the BT data and threshold it to find plume
    bt_data = np.array(scn2[procband].data)
    mask = np.where(bt_data < bt_thresh, 1, 0)
    
    # Erode edges and fill holes in mask
    mask = binary_erosion(mask)
    mask = binary_fill_holes(mask)
    
    # Now label it into objects
    labels = label(mask)
    shaper = labels.shape
    
    # Find the object near the middle of the scene, which will be the plume
    c_x = np.round(shaper[0]/2).astype(int)
    c_y = np.round(shaper[1]/2).astype(int)
    c_pt = np.median(labels[c_x-dpix:c_x+dpix, c_y-dpix:c_y+dpix])
    labels2 = np.where(labels==c_pt, 1, 0)
    
    # Compute the height and pressure of each pixel    
    out_height, out_pres = BTH.est_height_and_pressure(bt_data, ecm_dt, labels2)
    
    # Save the results
    scn2['labels'] = scn2[procband].copy()
    scn2['labels'].data = labels2
    scn2['height'] = scn2[procband].copy()
    scn2['height'].data = out_height
    scn2['pres'] = scn2[procband].copy()
    scn2['pres'].data = out_pres
    scn2.save_dataset('labels', base_dir=label_subdir, writer='simple_image')
    scn2.save_dataset('height', base_dir=height_subdir, dtype=np.float32, enhance=False)
    scn2.save_dataset('pres', base_dir=pres_subdir, dtype=np.float32, enhance=False)
    scn2.save_dataset(procband, base_dir=bt_subdir, dtype=np.float32, enhance=False)

  0%|          | 0/83 [00:00<?, ?it/s]